In [11]:
from __future__ import unicode_literals #you are building a byte string that holds UTF-8 encoded bytes.
                                        #With the string you are building a unicode string.
import youtube_dl #Command-line program to download videos from YouTube.com and other video sites


ydl_opts = {
    'format': 'bestaudio/best',     #getting the best format
    'extractaudio':True,            #only extract audio
    'audioformat':'wav',            #save audio in .wav format-- easy to convert as most recordings are in .wav
    'outtmpl':'%(id)s.%(ext)s',     #name the file as the ID of the video
    'noplaylist':True,              #no playlist download
    'nocheckcertificate':True,       
    'postprocessors': [{
        'key': 'FFmpegExtractAudio', #asks to extract audio
        'preferredcodec': 'wav',     #file type
        'preferredquality': '192',   #quality index
    }],
}
with youtube_dl.YoutubeDL(ydl_opts) as ydl:
    ydl.download(['https://www.youtube.com/watch?v=cO3QvioMESo'])

[youtube] cO3QvioMESo: Downloading webpage
[youtube] Downloading just video cO3QvioMESo because of --no-playlist
[youtube] cO3QvioMESo: Downloading MPD manifest
[dashsegments] Total fragments: 100
[download] Destination: cO3QvioMESo.m4a
[download] 100% of 15.18MiB in 00:11                    
[ffmpeg] Correcting container in "cO3QvioMESo.m4a"
[ffmpeg] Destination: cO3QvioMESo.wav
Deleting original file cO3QvioMESo.m4a (pass -k to keep)


In [12]:
import os                #provides functions for interacting with the operating system
import speech_recognition as sr     #library to perform speech recognition

from pydub import AudioSegment      #, if there are several overlapping sounds in this AudioSegment,this 
                                     #method will return one AudioSegment object for each of those sounds. 
from pydub.silence import split_on_silence     #if there is silence in the vid, then we will split the sound

r = sr.Recognizer()           # a function that splits the audio file into chunks
                                    # and applies speech recognition
                                
def get_large_audio_transcription(path):
    """
    Splitting the large audio file into chunks
    and apply speech recognition on each of these chunks
    """
    
    sound = AudioSegment.from_wav(path)   # open the audio file using pydub
    
    fh = open("converted.txt", "w+")  #open the file to write
    chunks = split_on_silence(sound,   # split audio sound where silence is 500 miliseconds or more and get chunks
       
        min_silence_len = 500,    # experiment with this value for your target audio file
        
        silence_thresh = sound.dBFS-14,  # adjust this per requirement
        
        keep_silence=500,   # keep the silence for 1 second, adjustable as well
    )
    folder_name = "audio-chunks"     # create a directory to store the audio chunks
   
    if not os.path.isdir(folder_name):    # asking os to create such foldername if not present and passing the text 
        os.mkdir(folder_name)
    whole_text = ""
    
    for i, audio_chunk in enumerate(chunks, start=1):     # export audio chunk and save it in
                                                         # the `folder_name` directory.
        
        chunk_filename = os.path.join(folder_name, f"chunk{i}.wav")
        audio_chunk.export(chunk_filename, format="wav")       # recognize the chunk
       
        with sr.AudioFile(chunk_filename) as source:
            audio_listened = r.record(source)               # try converting it to text
            
            try:
                text = r.recognize_google(audio_listened)
                fh.write(text+". ")      #writing in the .txt file
            except sr.UnknownValueError as e:
                print("Error:", str(e))
            else:
                text = f"{text.capitalize()}. "
                print(chunk_filename, ":", text)
                whole_text += text
                                         # return the text for all chunks detected
    return whole_text

In [13]:
path = "cO3QvioMESo.wav"
text = get_large_audio_transcription(path)
print("\nFull text:", text)

audio-chunks\chunk1.wav : I'm heather noelis and i was the first machine learning engineer hired to be on the ait mat. 
audio-chunks\chunk2.wav : T-mobile. 
audio-chunks\chunk3.wav : Can i talk to you a little bit about how we actually use are in production at t-mobile in the steps that you can do to do it as well because i know that nobody loves handing off their beautiful code to somebody else to just rewrite in python. 
audio-chunks\chunk4.wav : And so what do we actually do our goal is not to build a chatbot when you hear lots of organizations talk about doing natural language processing in real time you think chatbots. 
audio-chunks\chunk5.wav : Pergolas to essentially build a tool that the customer can interact or that the agent can interact with to help customers. 
audio-chunks\chunk6.wav : So i have a little screenshot of a sample of a tool that we have that. 
audio-chunks\chunk7.wav : If somebody says hello i'm traveling to europe and i need to make sure my phone works when yo

In [14]:
from pymongo import MongoClient
import gridfs

client = MongoClient()
db = client.test_database  # use a database called "test_database"
collection = db.files   # and inside that DB, a collection called "files"

f = open('converted.txt')  # open a file
text = f.read()    # read the entire contents, should be UTF-8 text

# build a document to be inserted
text_file_doc = {"file_name": "converted.txt", "contents" : text }
# insert the contents into the "file" collection
collection.insert(text_file_doc)

C:\Users\Shreya Gokhe\Anaconda3\lib\site-packages\ipykernel_launcher.py:14: DeprecationWarning: insert is deprecated. Use insert_one or insert_many instead.
  


ObjectId('5f6d3585eebd3cf2661bd617')

In [7]:
import nltk
nltk.download()

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


True

Using TF-IDF algorithm from NLTK

In [21]:
sentences = sent_tokenize(text) # NLTK function
total_documents = len(sentences)

NameError: name 'sent_tokenize' is not defined

In [22]:
def _create_frequency_matrix(sentences):
    frequency_matrix = {}
    stopWords = set(stopwords.words("english"))
    ps = PorterStemmer()

    for sent in sentences:
        freq_table = {}
        words = word_tokenize(sent)
        for word in words:
            word = word.lower()
            word = ps.stem(word)
            if word in stopWords:
                continue

            if word in freq_table:
                freq_table[word] += 1
            else:
                freq_table[word] = 1

        frequency_matrix[sent[:15]] = freq_table

    return frequency_matrix

In [23]:
def _create_tf_matrix(freq_matrix):
    tf_matrix = {}

    for sent, f_table in freq_matrix.items():
        tf_table = {}

        count_words_in_sentence = len(f_table)
        for word, count in f_table.items():
            tf_table[word] = count / count_words_in_sentence

        tf_matrix[sent] = tf_table

    return tf_matrix

In [24]:
def _create_documents_per_words(freq_matrix):
    word_per_doc_table = {}

    for sent, f_table in freq_matrix.items():
        for word, count in f_table.items():
            if word in word_per_doc_table:
                word_per_doc_table[word] += 1
            else:
                word_per_doc_table[word] = 1

    return word_per_doc_table

In [25]:
def _create_idf_matrix(freq_matrix, count_doc_per_words, total_documents):
    idf_matrix = {}

    for sent, f_table in freq_matrix.items():
        idf_table = {}

        for word in f_table.keys():
            idf_table[word] = math.log10(total_documents / float(count_doc_per_words[word]))

        idf_matrix[sent] = idf_table

    return idf_matrix

In [26]:
def _create_tf_idf_matrix(tf_matrix, idf_matrix):
    tf_idf_matrix = {}

    for (sent1, f_table1), (sent2, f_table2) in zip(tf_matrix.items(), idf_matrix.items()):

        tf_idf_table = {}

        for (word1, value1), (word2, value2) in zip(f_table1.items(),
                                                    f_table2.items()):  # here, keys are the same in both the table
            tf_idf_table[word1] = float(value1 * value2)

        tf_idf_matrix[sent1] = tf_idf_table

    return tf_idf_matrix

In [27]:
def _score_sentences(tf_idf_matrix) -> dict:
    """
    score a sentence by its word's TF
    Basic algorithm: adding the TF frequency of every non-stop word in a sentence divided by total no of words in a sentence.
    :rtype: dict
    """

    sentenceValue = {}

    for sent, f_table in tf_idf_matrix.items():
        total_score_per_sentence = 0

        count_words_in_sentence = len(f_table)
        for word, score in f_table.items():
            total_score_per_sentence += score

        sentenceValue[sent] = total_score_per_sentence / count_words_in_sentence

    return sentenceValue

In [28]:
def _find_average_score(sentenceValue) -> int:
    """
    Find the average score from the sentence value dictionary
    :rtype: int
    """
    sumValues = 0
    for entry in sentenceValue:
        sumValues += sentenceValue[entry]

    # Average value of a sentence from original summary_text
    average = (sumValues / len(sentenceValue))

    return average

In [29]:
def _generate_summary(sentences, sentenceValue, threshold):
    sentence_count = 0
    summary = ''

    for sentence in sentences:
        if sentence[:15] in sentenceValue and sentenceValue[sentence[:15]] >= (threshold):
            summary += " " + sentence
            sentence_count += 1

    return summary

In [30]:
import math

from nltk import sent_tokenize, word_tokenize, PorterStemmer
from nltk.corpus import stopwords    
    
'''
We already have a sentence tokenizer, so we just need 
to run the sent_tokenize() method to create the array of sentences.
'''
# 1 Sentence Tokenize
sentences = sent_tokenize(text)
total_documents = len(sentences)
#print(sentences)

# 2 Create the Frequency matrix of the words in each sentence.
freq_matrix = _create_frequency_matrix(sentences)
#print(freq_matrix)

'''
Term frequency (TF) is how often a word appears in a document, divided by how many words are there in a document.
'''
# 3 Calculate TermFrequency and generate a matrix
tf_matrix = _create_tf_matrix(freq_matrix)
#print(tf_matrix)

# 4 creating table for documents per words
count_doc_per_words = _create_documents_per_words(freq_matrix)
#print(count_doc_per_words)

'''
Inverse document frequency (IDF) is how unique or rare a word is.
'''
# 5 Calculate IDF and generate a matrix
idf_matrix = _create_idf_matrix(freq_matrix, count_doc_per_words, total_documents)
#print(idf_matrix)

# 6 Calculate TF-IDF and generate a matrix
tf_idf_matrix = _create_tf_idf_matrix(tf_matrix, idf_matrix)
#print(tf_idf_matrix)

# 7 Important Algorithm: score the sentences
sentence_scores = _score_sentences(tf_idf_matrix)
#print(sentence_scores)

# 8 Find the threshold
threshold = _find_average_score(sentence_scores)
#print(threshold)

# 9 Important Algorithm: Generate the summary
summary = _generate_summary(sentences, sentence_scores, 1.3 * threshold)
print(summary)

ZeroDivisionError: division by zero

### Using  cosine similarity method of NLTK
#### cosine similarity: used to determine how similar the documents are irrespective of their size.
#### Mathematically, it measures the cosine of the angle between two vectors projected in a multi-dimensional space. In this context, the two vectors are arrays containing the word counts of two documents.

In [34]:
from nltk.corpus import stopwords      #Stopwords are the English words which does not add much meaning to a sentence.
from nltk.cluster.util import cosine_distance 
import numpy as np

import networkx as nx        #NetworkX is a Python library for studying graphs and networks
 
def read_article(file_name):
    file = open(file_name, "r")
    filedata = file.readlines()
    article = filedata[0].split(". ")
    sentences = []

    for sentence in article:
        print(sentence)
        sentences.append(sentence.replace("[^a-zA-Z]", " ").split(" "))
    sentences.pop() 
    
    return sentences

def sentence_similarity(sent1, sent2, stopwords=None):
    if stopwords is None:
        stopwords = []
 
    sent1 = [w.lower() for w in sent1]
    sent2 = [w.lower() for w in sent2]
 
    all_words = list(set(sent1 + sent2))
 
    vector1 = [0] * len(all_words)
    vector2 = [0] * len(all_words)
 
    # build the vector for the first sentence
    for w in sent1:
        if w in stopwords:
            continue
        vector1[all_words.index(w)] += 1
 
    # build the vector for the second sentence
    for w in sent2:
        if w in stopwords:
            continue
        vector2[all_words.index(w)] += 1
 
    return 1 - cosine_distance(vector1, vector2)
 
def build_similarity_matrix(sentences, stop_words):
    # Create an empty similarity matrix
    similarity_matrix = np.zeros((len(sentences), len(sentences)))
 
    for idx1 in range(len(sentences)):
        for idx2 in range(len(sentences)):
            if idx1 == idx2: #ignore if both are same sentences
                continue 
            similarity_matrix[idx1][idx2] = sentence_similarity(sentences[idx1], sentences[idx2], stop_words)

    return similarity_matrix


def generate_summary(file_name, top_n=5):
    stop_words = stopwords.words('english')
    summarize_text = []
    fh = open("summary.txt", "w+")
    
    # Step 1 - Read text and split it
    sentences =  read_article(file_name)

    # Step 2 - Generate Similary Martix across sentences
    sentence_similarity_martix = build_similarity_matrix(sentences, stop_words)

    # Step 3 - Rank sentences in similarity martix
    np.seterr(divide='ignore', invalid='ignore')
    sentence_similarity_graph = nx.from_numpy_array(sentence_similarity_martix)
    scores = nx.pagerank_numpy(sentence_similarity_graph)

    # Step 4 - Sort the rank and pick top sentences
    ranked_sentence = sorted(((scores[i],s) for i,s in enumerate(sentences)), reverse=True)    
    print("Indexes of top ranked_sentence order are ", ranked_sentence)    

    for i in range(top_n):
        summarize_text.append(" ".join(ranked_sentence[i][1]))
        fh.write(str(summarize_text).strip('[').strip(']'))
    # Step 5 - Ofcourse, output the summarize text
    print("Summarize Text: \n", ". ".join(summarize_text))

     

In [35]:
generate_summary("converted.txt", 2)

I'm Heather noelis and I was the first machine learning engineer hired to be on the AIT mat
T-Mobile
can I talk to you a little bit about how we actually use are in production at T-Mobile in the steps that you can do to do it as well because I know that nobody loves handing off their beautiful code to somebody else to just rewrite in Python
and so what do we actually do our goal is not to build a chatbot when you hear lots of organizations talk about doing natural language processing in real time you think chatbots
pergolas to essentially build a tool that the customer can interact or that the agent can interact with to help customers
so I have a little screenshot of a sample of a tool that we have that
if somebody says hello I'm traveling to Europe and I need to make sure my phone works when you run it through an intent model we say you're talking about unlocked so let's go get some account information about this individual that the agent will need to help you and then let's do an inf

In [36]:
from pymongo import MongoClient
import gridfs

client = MongoClient()
db = client.test_database  # use a database called "test_database"
collection = db.files   # and inside that DB, a collection called "files"

f = open('summary.txt')  # open a file
text = f.read()    # read the entire contents, should be UTF-8 text

# build a document to be inserted
text_file_doc = {"file_name": "summary.txt", "contents" : text }
# insert the contents into the "file" collection
collection. insert(text_file_doc)

C:\Users\Shreya Gokhe\Anaconda3\lib\site-packages\ipykernel_launcher.py:14: DeprecationWarning: insert is deprecated. Use insert_one or insert_many instead.
  


ObjectId('5f6d390eeebd3cf2661bd61b')

In [37]:
text_file_d = collection.find_one({"file_name": "summary.txt", "contents" : text})
print(text_file_d['contents'])

"do you have the image which is after somebody builds that dockerfile and sets up the computer like you want that's the image and when you run it that's the container so people will throw these words that you will not but they're all really the same things just in different forms of the process""do you have the image which is after somebody builds that dockerfile and sets up the computer like you want that's the image and when you run it that's the container so people will throw these words that you will not but they're all really the same things just in different forms of the process", "containers are cheaper there is less computational overhead cuz you don't need to use a whole computer every time there's super trendy so people think that you're doing really cutting-edge technology when you talk about it they're made to crash and be restarted so every time that you restart your computer it takes forever to restart a Docker container"


In [38]:
from gtts import gTTS 
import os 
import pyttsx3 

In [39]:
# The text that you want to convert to audio 
  
# Language in which you want to convert 
language = 'en'

s = str(text_file_d['contents']).strip('"')    
# Passing the text and language to the engine,  
# here we have marked slow=False. Which tells  
# the module that the converted audio should  
# have a high speed 
myobj = gTTS(text=s, lang=language) 
  
# Saving the converted audio in a mp3 file named 
# welcome  
myobj.save("summ_speech.mp3") 
  
# Playing the converted file 
os.system("summ_spe.mp3") 

1